In [1]:
import pandas as pd
import sqlite3
import glob
import os

In [2]:
eventTypeTableFilenames = [fn for fn in glob.glob('event_*.csv') 
                           if not fn == "event_map_type.csv" and not fn == "event_object.csv"]
objectTypeTableFilenames =  [fn for fn in glob.glob('object_*.csv') 
                             if not fn == "object_map_type.csv" and not fn == "object_object.csv"]

TABLES = dict()

TABLES["event"] = pd.read_csv("event.csv", sep=";")
TABLES["event_map_type"] = pd.read_csv("event_map_type.csv", sep=";")
TABLES["event_object"] = pd.read_csv("event_object.csv", sep=";")
TABLES["object"] = pd.read_csv("object.csv", sep=";")
TABLES["object_object"] = pd.read_csv("object_object.csv", sep=";")
TABLES["object_map_type"] = pd.read_csv("object_map_type.csv", sep=";")

for fn in eventTypeTableFilenames:
    table_name = fn.split(".")[0]
    table = pd.read_csv(fn, sep=";")
    TABLES[table_name] = table
    
for fn in objectTypeTableFilenames:
    table_name = fn.split(".")[0]
    table = pd.read_csv(fn, sep=";")
    TABLES[table_name] = table

In [3]:
sql_path = "CargoPickup_IoT.sqlite"
if os.path.exists(sql_path):
    os.remove(sql_path)

In [4]:
conn = sqlite3.connect(sql_path)
for tn, df in TABLES.items():
    df.to_sql(tn, conn, index=False)
conn.close()

In [5]:
# Connect to your SQLite database
conn = sqlite3.connect("CargoPickup_IoT.sqlite")
cursor = conn.cursor()

# Create a new table without duplicates
cursor.execute('''CREATE TABLE IF NOT EXISTS object_object_no_duplicates AS
                  SELECT DISTINCT * FROM object_object''')

# Drop the original table
cursor.execute('DROP TABLE IF EXISTS object_object')

# Rename the new table to the original table name
cursor.execute('ALTER TABLE object_object_no_duplicates RENAME TO object_object')

# Commit changes and close the connection
conn.commit()
conn.close()

In [6]:
import pm4py

ocel = pm4py.read_ocel2_sqlite("CargoPickup_IoT.sqlite")
ocel

C:\Users\n9810200\Anaconda3\lib\site-packages\pm4py\objects\ocel\importer\sqlite\variants\ocel20.py:70: UserWarning: There are unsatisfied OCEL 2.0 constraints in the given relational database: ['const_2_existence_object_type_tables_map_obj_types', 'const_3_existence_event_type_tables_map_ev_types', 'const_14_primary_key_object_event_map_type_tables', 'const_15_primary_key_object_event_tables', 'const_16_primary_key_event_object_table', 'const_17_primary_key_object_object_table', 'const_18_primary_key_event_type_spec_tables', 'const_19_foreign_key_event', 'const_20_foreign_key_object', 'const_21_foreign_key_event_object', 'const_22_foreign_key_object_object', 'const_23_foreign_key_event_type_specific', 'const_24_foreign_key_object_type_specific']
  warnings.warn("There are unsatisfied OCEL 2.0 constraints in the given relational database: "+str(unsatisfied))
C:\Users\n9810200\Anaconda3\lib\site-packages\pm4py\objects\log\util\dataframe_utils.py:183: SettingWithCopyWarning: 
A value is 

Object-Centric Event Log (number of events: 3611, number of objects: 80, number of activities: 13, number of object types: 4, events-objects relationships: 3621)
Activities occurrences: {'Assign Truck': 491, 'Enter the port': 491, 'Weigh the Empty Truck': 491, 'Check Empty Truck Weight Abnormality': 491, 'Fail to Weigh': 300, 'Arrive at the Silo': 191, 'Determine the Continuance of the Pickup': 191, 'Load Truck': 191, 'Weigh the Loaded Truck': 191, 'Input the Tally Sheet': 191, 'Print the Weighing Ticket': 191, 'Evaluate the Truck Exit': 191, 'Lodge Pickup Plan': 10}
Object types occurrences (number of objects): {'Truck': 50, 'Cargo': 10, 'Pickup Plan': 10, 'Silo': 10}
Please use <THIS>.get_extended_table() to get a dataframe representation of the events related to the objects.

In [7]:
ocel.get_summary()

"Object-Centric Event Log (number of events: 3611, number of objects: 80, number of activities: 13, number of object types: 4, events-objects relationships: 3621)\nActivities occurrences: {'Assign Truck': 491, 'Enter the port': 491, 'Weigh the Empty Truck': 491, 'Check Empty Truck Weight Abnormality': 491, 'Fail to Weigh': 300, 'Arrive at the Silo': 191, 'Determine the Continuance of the Pickup': 191, 'Load Truck': 191, 'Weigh the Loaded Truck': 191, 'Input the Tally Sheet': 191, 'Print the Weighing Ticket': 191, 'Evaluate the Truck Exit': 191, 'Lodge Pickup Plan': 10}\nObject types occurrences (number of objects): {'Truck': 50, 'Cargo': 10, 'Pickup Plan': 10, 'Silo': 10}\nPlease use <THIS>.get_extended_table() to get a dataframe representation of the events related to the objects."

### Basic Statistics

#### E2O table

In [8]:
e2o_df = pd.read_csv('event_object.csv')
e2o_df = e2o_df['ocel_event_id;ocel_object_id;ocel_qualifier'].str.split(';', expand=True)
e2o_df.columns = ['ocel_event_id', 'ocel_object_id', 'ocel_qualifier']

In [9]:
len(e2o_df)

3621

#### O2O table

In [10]:
o2o_df = pd.read_csv('object_object.csv')
o2o_df = o2o_df['ocel_source_id;ocel_target_id;ocel_qualifier;ocel_time'].str.split(';', expand=True)
o2o_df.columns = ['ocel_source_id', 'ocel_target_id', 'ocel_qualifier','ocel_time']

In [11]:
len(o2o_df)

883

#### IoT2E table

In [12]:
e2IoTo_df = pd.read_csv('event_IoTobject.csv')
e2IoTo_df = e2IoTo_df['ocel_event_id;ocel_IoT_object_id;ocel_qualifier'].str.split(';', expand=True)
e2IoTo_df.columns = ['ocel_event_id', 'ocel_IoT_object_id', 'ocel_qualifier']

In [13]:
len(e2IoTo_df)

2619

#### IoT2O table

In [14]:
o2IoTo_df = pd.read_csv('object_IoTobject.csv')
o2IoTo_df = o2IoTo_df['ocel_IoTobject_id;ocel_object_id;ocel_qualifier;ocel_time'].str.split(';', expand=True)
o2IoTo_df.columns = ['ocel_IoTobject_id', 'ocel_object_id', 'ocel_qualifier','ocel_time']

In [15]:
len(o2IoTo_df)

1637